In [ ]:
###REMEMBER TO CLEAR os.chdir() AND RESTART KERNEL AND CLEAR OUTPUT 

###FOR EXAMPLE: AAPL
#https://investor.apple.com/sec-filings/default.aspx
#10Q and 10K in whichStock directory   

In [ ]:
from urllib.request import Request, urlopen

import os
import requests
import re
import json
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
#get API key file contents and connect to FMP API
#os.chdir('~')

FMP_API_KEY = ""
BASE_URL = ""

with open('FMP_API.txt', 'r') as file: 
    for line in file: 
        if line.startswith('FMP_API_KEY'): 
            FMP_API_KEY = line.split('"')[1]
        elif line.startswith('BASE_URL'): 
            BASE_URL = line.split('"')[1]

ticker = input("Enter ticker symbol for Buffett DCA Analysis: ")

In [ ]:
# income statements calls
# without limit parameter in URL: 
annual_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?apikey=' + FMP_API_KEY)).json())
quarterly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&apikey=' + FMP_API_KEY)).json())

#used in per share earnings formulation
shares_call = pd.DataFrame((requests.get(BASE_URL + '/api/v4/shares_float?symbol=' + ticker + '&apikey=' + FMP_API_KEY)).json())

# balance sheet calls, without limit parameter in URL: 
annual_balance_sheet = pd.DataFrame((requests.get(BASE_URL + '/api/v3/balance-sheet-statement/' + ticker + '?apikey=' + FMP_API_KEY)).json())
quarterly_balance_sheet = pd.DataFrame((requests.get(BASE_URL + '/api/v3/balance-sheet-statement/' + ticker + '?period=quarter&apikey=' + FMP_API_KEY)).json())

In [ ]:
annual_balance_sheet.head()

In [ ]:
annual_income_stmt.head()

In [ ]:
quarterly_income_stmt.columns

In [ ]:
#[INCOME STATEMENT][REVENUE][COST OF GOODS SOLD]
#   calculate gross profit, annually and monthly
#   WB formula:   NET REVENUE = TOTAL REVENUE - COGS
#   FMP formula:  REVENUE - COST OF REVENUE

annual_metrics = pd.DataFrame()
annual_metrics['Ticker'], annual_metrics['Frequency'], annual_metrics['Year'], annual_metrics["Filing Date"], annual_metrics["Gross Profit"] = [ticker, '', annual_income_stmt['calendarYear'], annual_income_stmt["fillingDate"], annual_income_stmt['revenue'] - annual_income_stmt['costOfRevenue']]
annual_metrics['Ticker'] = ticker
annual_metrics['Frequency'] = 'Annually'

#annual_metrics

quarterly_metrics = pd.DataFrame()
quarterly_metrics['Ticker'], quarterly_metrics['Frequency'], quarterly_metrics['Year'], quarterly_metrics['Quarter'], quarterly_metrics["Filing Date"], quarterly_metrics["Gross Profit"] = [ticker, '', quarterly_income_stmt['calendarYear'], quarterly_income_stmt['period'], quarterly_income_stmt['fillingDate'], quarterly_income_stmt['revenue'] - quarterly_income_stmt['costOfRevenue']]
quarterly_metrics['Ticker'] = ticker
quarterly_metrics['Frequency'] = 'Quarterly'

#quarterly_metrics


In [ ]:
#[INCOME STATEMENT][COST OF GOODS SOLD]
#   store COGS value in metrics table, annually and quarterly
annual_metrics['COGS'] = annual_income_stmt['costOfRevenue']

quarterly_metrics['COGS'] = quarterly_income_stmt['costOfRevenue']

In [ ]:
#annual_metrics.head()
#quarterly_metrics.head()

In [ ]:
#[INCOME STATEMENT][REVENUE][COST OF GOODS SOLD]
#   calculate gross profit margin, annually and monthly
#   WB formula:   GROSS PROFIT MARGIN = GROSS PROFIT / TOTAL REVENUE
#   FMP formula:  GROSS PROFIT MARGIN = GROSS PROFIT / REVENUE

#1 IN DATAFRAME NAME SINCE FIRST TIME MERGING RAW AND CALCULATED DATA
annual_joined_1 = annual_income_stmt.merge(annual_metrics, how = 'inner', left_on = ['fillingDate', 'calendarYear'], right_on = ['Filing Date', 'Year'])
quarterly_joined_1 = quarterly_income_stmt.merge(quarterly_metrics, how = 'inner', left_on = ['fillingDate', 'calendarYear', 'period'], right_on = ['Filing Date', 'Year', 'Quarter'])

#calculations for GPM 
annual_metrics['Gross Profit Margin'] = (annual_joined_1['Gross Profit'] / annual_joined_1['revenue'])

quarterly_metrics['Gross Profit Margin'] = quarterly_joined_1['Gross Profit'] / quarterly_joined_1['revenue']

In [ ]:
#[INCOME STATEMENT][OPERATING EXPENSES][GROSS PROFIT]
#   calculate operating profit or loss, annually and monthly
#   WB formula: GROSS PROFIT - TOTAL OPERATING EXPENSES
#   FMP formula: GROSS PROFIT - OPERATING EXPENSES

#calculations for operating P/L
annual_metrics['Operating P/L'] = annual_joined_1['Gross Profit'] - annual_joined_1['operatingExpenses']

quarterly_metrics['Operating P/L'] = quarterly_joined_1['Gross Profit'] - quarterly_joined_1['operatingExpenses']

In [ ]:
#[INCOME STATEMENT][SELLING, GENERAL, ADMINISTRATIVE EXPENSES][GROSS PROFIT]
#   calculate SGA percentage as a percentage of gross profit, annually and monthly
#   WB formula: (1) SGA amount (2) SGA / gross profit
#   FMP formula: sellingGeneralAndAdministrativeExpenses

annual_metrics['SGA'] = annual_joined_1['sellingGeneralAndAdministrativeExpenses']
annual_metrics['SGA by Gross Profit'] = (annual_joined_1['sellingGeneralAndAdministrativeExpenses'] / annual_joined_1['Gross Profit'])

quarterly_metrics['SGA'] = quarterly_joined_1['sellingGeneralAndAdministrativeExpenses']
quarterly_metrics['SGA by Gross Profit'] = (quarterly_joined_1['sellingGeneralAndAdministrativeExpenses'] / quarterly_joined_1['Gross Profit'])

In [ ]:
#[INCOME STATEMENT][RESEARCH AND DEVELOPMENT][GROSS PROFIT]
#   calculate R&D percentage as a percentage of gross profit, annually and monthly
#   WB formula: (1) R&D amount (2) R&D / gross profit
#   FMP formula: researchAndDevelopmentExpenses

annual_metrics['R&D'] = annual_joined_1['researchAndDevelopmentExpenses']
annual_metrics['R&D by Gross Profit'] = (annual_joined_1['researchAndDevelopmentExpenses'] / annual_joined_1['Gross Profit'])

quarterly_metrics['R&D'] = quarterly_joined_1['researchAndDevelopmentExpenses']
quarterly_metrics['R&D by Gross Profit'] = (quarterly_joined_1['researchAndDevelopmentExpenses'] / quarterly_joined_1['Gross Profit'])

In [ ]:
#[INCOME STATEMENT][DEPRECIATION][OPERATING PROFIT]
#   calculate SGA percentage as a percentage of gross profit, annually and monthly
#   WB formula: (1) Depreciation amount (2) Depreciation / gross profit
#   FMP formula: depreciationAndAmortization
#   also include EBITDA

annual_metrics['EBITDA'] = annual_joined_1['ebitda']
annual_metrics['Depreciation'] = annual_joined_1['depreciationAndAmortization']
annual_metrics['Depreciation by Operating Profit'] = (annual_joined_1['depreciationAndAmortization'] / annual_joined_1['Gross Profit'])

quarterly_metrics['ebitda'] = quarterly_joined_1['ebitda']
quarterly_metrics['Depreciation'] = quarterly_joined_1['depreciationAndAmortization']
quarterly_metrics['Depreciation by Operating Profit'] = (quarterly_joined_1['depreciationAndAmortization'] / quarterly_joined_1['Gross Profit'])

In [ ]:
#[INCOME STATEMENT][INTEREST EXPENSE][OPERATING P/L]
#   calculate Interest Expense percentage as a percentage of operating P/L, annually and monthly
#   WB formula: interest expense / operating income
#   FMP formula: interestExpense

annual_metrics['Interest Expense'] = annual_joined_1['interestExpense']
annual_metrics['Interest Expense by Operating P/L'] = (annual_joined_1['interestExpense'] / annual_metrics['Operating P/L'])

quarterly_metrics['Interest Expense'] = quarterly_joined_1['interestExpense']
quarterly_metrics['Interest Expense by Operating P/L'] = (quarterly_joined_1['interestExpense'] / quarterly_metrics['Operating P/L'])

In [ ]:
#[INCOME STATEMENT][GAIN/(LOSS) SALE ASSETS, OTHER]
#   not used, not tracked

In [ ]:
#[INCOME STATEMENT][INCOME BEFORE TAX]
#   report as is
#   FMP formula: incomeBeforeTax

annual_metrics['Income Before Taxes'] = annual_joined_1['incomeBeforeTax']

quarterly_metrics['Income Before Taxes'] = quarterly_joined_1['incomeBeforeTax']

In [ ]:
#[INCOME STATEMENT][INCOME TAXES PAID] AKA [NET INCOME]
#   [INCOME BEFORE TAX] - ([INCOME BEFORE TAX] * 0.35) != [INCOME TAXES PAID] ==> BAD
#   FMP formula: incomeBeforeTax, incomeTaxExpense
#   Need to determine logic for if GOOD or BAD, till then differences are used

annual_metrics['Taxes Paid'] = annual_joined_1['incomeTaxExpense']
annual_metrics['ifTaxesPaidAreCorrect'] = (annual_joined_1['incomeBeforeTax'] - ((annual_joined_1['incomeBeforeTax'] * 0.35)))
annual_metrics['Difference between Taxes to Pay and Actual Taxes Paid'] = annual_metrics['ifTaxesPaidAreCorrect'] - annual_metrics['Taxes Paid']

quarterly_metrics['Taxes Paid'] = annual_joined_1['incomeTaxExpense']
quarterly_metrics['ifTaxesPaidAreCorrect'] = (quarterly_joined_1['incomeBeforeTax'] - ((quarterly_joined_1['incomeBeforeTax'] * 0.35)))
quarterly_metrics['Difference between Taxes to Pay and Actual Taxes Paid'] = quarterly_metrics['ifTaxesPaidAreCorrect'] - annual_metrics['Taxes Paid']

In [ ]:
#[INCOME STATEMENT][NET EARNINGS]
#   FMP formula: netIncome = incomeBeforeTax - incomeTaxExpense

#revenue is also needed in this analysis (companies with DCA: high net earnings to total revenue ratio)

annual_metrics['Net Earnings'] = annual_joined_1['netIncome']
annual_metrics['Total Revenue'] = annual_joined_1['revenue']

quarterly_metrics['Net Earnings'] = quarterly_joined_1['netIncome']
quarterly_metrics['Total Revenue'] = quarterly_joined_1['revenue']

In [ ]:
#[INCOME STATEMENT][NET EARNINGS]
# per share earnings
#   FMP formula: netIncome / outstandingShares
#outstandingShares is from another API call
outstanding_shares = shares_call.iloc[0]['outstandingShares']

annual_metrics['Per-Share Earnings'] = annual_joined_1['netIncome'] / outstanding_shares

quarterly_metrics['Per-Share Earnings'] = quarterly_joined_1['netIncome'] / outstanding_shares

In [ ]:
#API response URLs with limit parameter in URL
#yearly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?limit=120&apikey=' + FMP_API_KEY)).json())
#monthly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&limit=120&apikey=' + FMP_API_KEY)).json())

#without limit parameter in URL: 
#yearly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?apikey=' + FMP_API_KEY)).json())
#monthly_income_stmt = pd.DataFrame((requests.get(BASE_URL + '/api/v3/income-statement/' + ticker + '?period=quarter&apikey=' + FMP_API_KEY)).json())

#response_from_API = yearly_income.json()
#df = pd.DataFrame(response_from_API)